# Rasterio

(bands, rows, columns)

what the hell is the *band cache*?

https://rasterio.readthedocs.io/en/latest/topics/index.html !? not in index


? verify crs and coord <-> pixel conversion functions and aviris

open in update mode? r+, w ?


how does rasterio memory use work? load to mem or stay on disk?

- rasterio depends on GDAL
- there is a command line interface `rio`

- https://rasterio.readthedocs.io/en/latest/ (an older set was at https://mapbox.github.io/rasterio/ but was removed)
- https://rasterio.groups.io/g/main support forum
- https://github.com/mapbox/rasterio
- https://github.com/mapbox/rasterio-cookbook
- https://sgillies.net/tags/rasterio.html is good for current news!
- https://github.com/sgillies/affine

see also shapely and fiona, maybe also cartopy, pyproj, pysal, etc.

this looks interesting https://github.com/atedstone/georaster https://georaster.readthedocs.io/en/latest/

todo: read up on this good ref https://chris35wills.github.io/courses/pydata_stack/

note: installing dev version caused a lot of packages to downgrade!

    conda install -c conda-forge/label/dev rasterio

## rasterio changes

https://sgillies.net/2018/07/13/rasterio-1-0-0.html

https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html

https://github.com/mapbox/rasterio/blob/master/CHANGES.txt

# rasterio objects

- `DatasetReader`
- `Band`
- `Window`
- `CRS`
- `Affine`
- ...

### DatasetReader

Let's focus on reading raster data, and ignore the idea of writing it for the time being.

Dataset is read into a `DatasetReader` object (older versions of Rasterio may create a `RasterReader`).

There is some GDAL object beneath?

[rasterio._base.DatasetBase](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html?highlight=meta#rasterio._base.DatasetBase) > [rasterio._io.DatasetReaderBase](https://rasterio.readthedocs.io/en/latest/api/rasterio._io.html#rasterio._io.DatasetReaderBase) > [rasterio.io.DatasetReader](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader)

#### DatasetReader Properties

* `meta`
* `profile`
* `count`
* `crs`
* descriptions?
* units?

 ## understanding rasterio
 
 Affine?
 
 Rasterio is confusing.
 
 The basic `rasterio` module is in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py)

The [rasterio._base module](https://rasterio.readthedocs.io/en/latest/api/rasterio._base.html) (defined in [`_base.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_base.pyx)) defines numpy-free base classes, including `DatasetBase`. Some methods include `get_crs()`, `get_transform()`, as well as `read_crs()` and `read_transform()` but they are just called by the corresponding `_get_*` methods to set the values of `_crs` and `_transform` if they are not already set.

__i think some need to be imported individually!__ e.g. rasterio.plot
and the underscore versions are some compiled thing

https://rasterio.readthedocs.io/en/latest/api/ lists modules and submodules but not all are available
 
- `compat`
- `coords`
- crs `_crs`
- drivers `_drivers` 
- `dtypes`
- `enums`
- `env`
- `errors`
- features `_features`
- fill `_fill`
- io `_io`?
- mask ?
- merge ?
- plot ?
- `profiles`
- sample ?
- `transform`
- `vfs`
- vrt ?
- warp `_warp`
- `windows`
 
`rasterio.io` is kind of available as `rasterio._io` but not certain


In [ ]:
import rasterio

In [ ]:
# Uncomment to examine rasterio package.
# dir(rasterio)
# help(rasterio)

questions and tasks:

- learn about masks

## opening a raster file

- [`rasterio.open()`](https://rasterio.readthedocs.io/en/latest/api/rasterio.html#rasterio.open), defined in [`__init__.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/__init__.py), returns a [`DatasetReader`](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html#rasterio.io.DatasetReader)
- `rasterio.band()` allows you to wrap a dataset and and one or more of its bands up into a `rasterio.Band`, which is really just a tuple:

        Band = namedtuple('Band', ['ds', 'bidx', 'dtype', 'shape'])

definitions and inheritance:

- `DatasetReader`, `MemoryFile`, etc. are part of the [rasterio.io module](https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html) (defined in [`io.py`](https://github.com/mapbox/rasterio/blob/master/rasterio/io.py), but nothing is really defined until `_io.pyx`).
- `DataSetReader` inherits from `rasterio._io.DatasetReaderBase`, `rasterio.windows.WindowMethodsMixin`, `rasterio.transform.TransformMethodsMixin`.
- The [rasterio._io module](https://rasterio.readthedocs.io/en/latest/api/rasterio._io.html) (defined in [`_io.pyx`](https://github.com/mapbox/rasterio/blob/master/rasterio/_io.pyx)) defines `DatasetReaderBase`, `MemoryFileBase`, `InMemoryRaster`, and some writer classes, also includes `read()`.

see also MemoryFile? for in-memory?

read https://github.com/mapbox/rasterio/issues/86 sometime!
and see https://rasterio.readthedocs.io/en/latest/topics/migrating-to-v1.html


In [ ]:
rasterio.band?

https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html

## reading files into memory?

[Readings Datasets](https://rasterio.readthedocs.io/en/latest/topics/reading.html)

`dataset.read()` (defined in `_io.pyx`) will read pixels into a numpy array. Without args it will read the entire dataset, or else specify which band or bands to read.

## plotting in rasterio

- https://rasterio.readthedocs.io/en/latest/topics/plotting.html [[old](https://mapbox.github.io/rasterio/topics/plotting.html)]
- https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html [[old](https://mapbox.github.io/rasterio/api/rasterio.plot.html)]

see https://github.com/mapbox/rasterio/blob/master/rasterio/plot.py

`rasterio.plot.show(source, with_bounds=True, contour=False, contour_label_kws=None, ax=None, title=None, transform=None, adjust='linear', **kwargs)`
is a wrapper for pyplot, I believe imshow but check?

source can be a Band or tuple (dataset, bdx), or array, or dataset in which case the first band is diaplayed unless colorinterp metadata is set - how?

TODO it's really unclear how to plot three bands from a multispectral image! seems possible if you create a band object

WTF is "rasterio band order"?

In [ ]:
#? from rasterio._